In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")

raw_path = "../data/NBA_Draft_ve.csv"
df = pd.read_csv(raw_path)
CURRENT_YEAR = 2025

df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [1]:
import pandas as pd
import numpy as np

raw_path = "../data/NBA_Draft_ve.csv"
df = pd.read_csv(raw_path)
CURRENT_YEAR = 2025



Normalizzazione dello storico

In [2]:
df = df.rename(columns={
    "Year": "DraftYear",
    "Rk": "Pick",
    "Yrs": "Seasons",
    "G": "Games"
})

In [3]:
df.head()

,DraftYear,Pick,Tm,Player,College,Seasons,Games,MP,PTS,TRB,...,3P%,FT%,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP
0,1947,1.0,PIT,Clifton McNeely,Texas Wesleyan University,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947,2.0,TRH,Glen Selbo,Wisconsin,1,13.0,NaN,42.0,NaN,...,NaN,0.759,NaN,3.2,NaN,1.8,-0.2,NaN,NaN,NaN
2,1947,3.0,BOS,Bulbs Ehlers,Purdue,2,99.0,NaN,800.0,NaN,...,NaN,0.618,NaN,8.1,NaN,1.8,3.0,NaN,NaN,NaN
3,1947,4.0,PRO,Walt Dropo,UConn,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1947,5.0,NYK,Dick Holub,Long Island University,1,48.0,NaN,504.0,NaN,...,NaN,0.633,NaN,10.5,NaN,0.8,4.2,NaN,NaN,NaN


In [4]:
# Valori mancanti per colonna
df.isna().sum()


DraftYear       0
Pick            1
Tm              1
Player          1
College         3
Seasons      4640
Games        4635
MP           4734
PTS          4635
TRB          4706
AST          4635
FG%          4643
3P%          6081
FT%          4727
MP.1         4734
PTS.1        4635
TRB.1        4706
AST.1        4635
WS           4635
WS/48        4735
BPM          5307
VORP         5306
dtype: int64

OSSERVAZIONe-> nel dataset ci sono valori nulli per le seguenti feature anagrafiche:
- Pick
- Tm
- Player
- Collage (3)
Questo è dovuto ad una multa per i Washington Bullets che nel 1997 hanno dovuto rinunciare ad una scelta. Per i restanti missing Values sulla feature collage, questo è dipeso dal fatto che i primi anni chi arrivava dall'estero non aveva un collage di appartenenza, gestito nel dataset inserendolo manualmente "Overseas (Greece)" perche il giocatore in questione proveniva dalla grecia, oppure il collage di appartenenza era sconosciuto, sostituito con valore "UNKNOWN".

Mentre i NaN su dati statistici sono giustificati dal fatto che quel determinato giocatore non ha mai esordito in NBA.

In [5]:
# Crea una colonna che marca la pick forfeitata
df["Forfeited"] = df["Player"].isna() & (df["DraftYear"] == 1997)

# Dataset “pulito” per le analisi sui giocatori
df = df[~df["Forfeited"]].copy()


In [6]:
#Gestione valori NaN sui college
df.loc[df["College"].isna() & (df["DraftYear"] < 1980), "College"] = "UNKNOWN"
df.loc[df["College"].isna() & (df["DraftYear"] >= 1980), "College"] = "Overseas (Greece)"


Divisione delle scelte

In [7]:
def pick_band(p):
    if pd.isna(p):
        return "Unknown"
    if p <= 10:
        return "Top10"
    elif p <= 30:
        return "FirstRound"
    else:
        return "SecondRound"

df["PickBand"] = df["Pick"].apply(pick_band)


Aggiunta di una nuova colonna per separare i giocatori che hanno esordito in nba con quelli che non hanno esordito in NBA

Parte complessa e lunga per capire come gestire la mole di dati e le varie motivazione per cui i valori sono mancanti, e gestire ogni caso separatamente

Riassunto della semantica dei valori:
Debut = 0
Seasons, Games e stats base = 0
WS, WS/48, BPM, VORP = 0 (mai giocato → nessun impatto)
Debut = 1
3P% = -1 → giocatore pre-1979, statistica non applicabile
3P% = 0 → dopo il 1979, ma 0% (o dato mancante trattato come 0)
WS / WS/48 / BPM / VORP = -100 → dato non calcolabile / non disponibile
WS / WS/48 / BPM / VORP qualsiasi altro numero (anche 0 o < 0) → valore reale
colonne WS_available, WS/48_available, BPM_available, VORP_available:
1 → valore reale presente
0 → era mancante ed è stato codificato con -100

In [8]:
import pandas as pd

# 1. Debut: 1 se ha giocato almeno 1 partita NBA, altrimenti 0
df["Debut"] = (df["Games"] > 0).astype(int)

# 2. Per chi NON ha debuttato: azzero solo le statistiche "classiche"
base_stats = [
    "Seasons", "Games",
    "MP", "PTS", "TRB", "AST",
    "FG%", "FT%",
    "MP.1", "PTS.1", "TRB.1", "AST.1"
]

df.loc[df["Debut"] == 0, base_stats] = 0

# 3. Gestione del tiro da 3 (3P%)
df["3P%"] = pd.to_numeric(df["3P%"], errors="coerce")

mask_after_1979 = df["DraftYear"] >= 1979
mask_before_1979 = df["DraftYear"] < 1979

# Dopo il 1979: dove manca, metto 0
df.loc[mask_after_1979, "3P%"] = df.loc[mask_after_1979, "3P%"].fillna(0)

# Prima del 1979: il dato non esiste proprio
df.loc[mask_before_1979, "3P%"] = -1

# 4. Statistiche avanzate: WS, WS/48, BPM, VORP
adv_cols = ["WS", "WS/48", "BPM", "VORP"]

# Converto le colonne avanzate a numeriche, così i NaN sono veri NaN
for col in adv_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# 4a. Per chi NON ha debuttato: metriche avanzate a 0 (nessun impatto)
df.loc[df["Debut"] == 0, adv_cols] = 0

# 4b. Creazione flag e imputazione -100 SOLO per chi ha debuttato ma il dato NON è calcolato
mask_debut = df["Debut"] == 1

for col in adv_cols:
    mask_missing = mask_debut & df[col].isna()          # <--- importante: cattura i veri mancanti
    df[col + "_available"] = (~mask_missing).astype(int) # 1 = dato reale, 0 = mancante
    df.loc[mask_missing, col] = -100                     # imputazione sentinella


In [ ]:
df.isna().sum()

DraftYear           0
Pick                0
Tm                  0
Player              0
College             0
Seasons             6
Games               0
MP                 99
PTS                 0
TRB                71
AST                 0
FG%                 8
3P%                 0
FT%                92
MP.1               99
PTS.1               0
TRB.1              71
AST.1               0
WS                  0
WS/48               0
BPM                 0
VORP                0
Forfeited           0
PickBand            0
Debut               0
WS_available        0
WS/48_available     0
BPM_available       0
VORP_available      0
dtype: int64

Per Valori che non esistono metto -100, altrimenti per season metto 1

In [10]:
# 1. SEASONS mancanti → se Debut=1 deve essere almeno 1
df["Seasons"] = df["Seasons"].fillna(1)

# 2. STATISTICHE TOTALI MANCANTI → -100 + flag
missing_total_cols = ["MP", "TRB", "FG%", "FT%"]

for col in missing_total_cols:
    mask_missing = df[col].isna()                          # flag basata sui dati ORIGINALI
    df[col + "_available"] = (~mask_missing).astype(int)   # 1 = disponibile, 0 = mancante
    df.loc[mask_missing, col] = -100                       # imputazione

# 3. STATISTICHE PER-GAME MANCANTI → -100 + flag
missing_pergame_cols = ["MP.1", "TRB.1"]

for col in missing_pergame_cols:
    mask_missing = df[col].isna()
    df[col + "_available"] = (~mask_missing).astype(int)
    df.loc[mask_missing, col] = -100


In [11]:
df.isna().sum()

DraftYear          0
Pick               0
Tm                 0
Player             0
College            0
Seasons            0
Games              0
MP                 0
PTS                0
TRB                0
AST                0
FG%                0
3P%                0
FT%                0
MP.1               0
PTS.1              0
TRB.1              0
AST.1              0
WS                 0
WS/48              0
BPM                0
VORP               0
Forfeited          0
PickBand           0
Debut              0
WS_available       0
WS/48_available    0
BPM_available      0
VORP_available     0
MP_available       0
TRB_available      0
FG%_available      0
FT%_available      0
MP.1_available     0
TRB.1_available    0
dtype: int64

In [12]:
# Conversione a numerico
df["DraftYear"] = pd.to_numeric(df["DraftYear"], errors="coerce")
df["Seasons"]   = pd.to_numeric(df["Seasons"],   errors="coerce")

df[["DraftYear", "Seasons"]] = df[["DraftYear", "Seasons"]].fillna(0)

# Calcolo dello stato SENZA creare una colonna EndYear
df["Status"] = (df["DraftYear"] + df["Seasons"] >= CURRENT_YEAR) \
                .map({True: "Active", False: "Retired"})

In [13]:
df.to_csv("../data/drafted_cleaned.csv", index=False)
